## SILVER TO GOLD LAYER

### Gold Layer - Trade History


In [1]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import Portfolio, global_path, logger

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [2]:
# Read the CSV file
try:
    df_trade_history = pd.read_csv(global_path.tradehistory_silver_file_path)

    # Convert 'datetime' to datetime type
    df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

    # Sort the DataFrame by 'datetime'
    df_trade_history = df_trade_history.sort_values(by="datetime")

    logger.info(
        f"Read SILVER Layer trade history data from: {global_path.tradehistory_silver_file_path}"
    )
except Exception as e:
    logger.error(
        f"Failed to read SILVER Layer trade history data due to error: {e}"
    )

2024-08-01T14:05:16Z - INFO - Read SILVER Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\TradeHistory\TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks and round necessary columns.


In [3]:
# Apply the trade logic to each row of the DataFrame
try:
    # Instantiate the Portfolio object
    portfolio = Portfolio()

    data = [
        portfolio.trade(row.to_dict()) for _, row in df_trade_history.iterrows()
    ]
    data += portfolio.check_expired_stocks()

    # Create a DataFrame from the processed data
    df_trade_history = pd.DataFrame(data)

    # Round the values in the columns to two decimal places
    df_trade_history = df_trade_history.round(2)
    

    df_trade_history = df_trade_history.fillna(None)

except Exception as e:
    logger.error(f"Failed to process trade history data due to error: {e}")

2024-08-01T14:05:17Z - ERROR - Failed to process trade history data due to error: Must specify a fill 'value' or 'method'.


### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [4]:
# Save the result as a CSV file
try:
    # Sort the DataFrame by 'segment', 'stock_name', and 'datetime'
    df_trade_history = df_trade_history.sort_values(
        by=["segment", "stock_name", "datetime"]
    )
    # Select relevant columns
    df_trade_history = df_trade_history[
        [
            "datetime",
            "exchange",
            "segment",
            "stock_name",
            "scrip_code",
            "expiry_date",
            "side",
            "quantity",
            "price",
            "amount",
            "holding_quantity",
            "avg_price",
            "holding_amount",
            "pnl_amount",
            "pnl_percentage"
        ]
    ]

    df_trade_history.to_csv(global_path.tradehistory_gold_file_path, index=None)
    logger.info(
        "GOLD Layer CSV file for trade history successfully created at:"
    )
    logger.info(global_path.tradehistory_gold_file_path.resolve())
    # Log the DataFrame info
    df_trade_history.info()
except Exception as e:
    logger.error(f"Failed to save GOLD Layer CSV file due to error: {e}")

2024-08-01T14:05:17Z - INFO - GOLD Layer CSV file for trade history successfully created at:
2024-08-01T14:05:17Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\TradeHistory\TradeHistory_data.csv


<class 'pandas.core.frame.DataFrame'>
Index: 176 entries, 2 to 93
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datetime          176 non-null    datetime64[ns]
 1   exchange          176 non-null    object        
 2   segment           176 non-null    object        
 3   stock_name        176 non-null    object        
 4   scrip_code        176 non-null    object        
 5   expiry_date       176 non-null    object        
 6   side              176 non-null    object        
 7   quantity          176 non-null    float64       
 8   price             176 non-null    float64       
 9   amount            176 non-null    float64       
 10  holding_quantity  176 non-null    float64       
 11  avg_price         176 non-null    float64       
 12  holding_amount    176 non-null    float64       
 13  pnl_amount        176 non-null    float64       
 14  pnl_percentage    176 non-null  